In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor

In [ ]:

training_data = datasets.MNIST(
    "data", 
    train=True, 
    download=True, 
    transform=ToTensor()
)

test_data = datasets.MNIST(
    "data", 
    train=False, 
    download=True, 
    transform=ToTensor() 
)

In [ ]:
batch_size = 128

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for X, y in train_dataloader:
    print(f"Shape of X[N, C, W, H]:{X.shape}")
    print(f"Shape of y:{y[0]}, {y.shape}, {y.dtype}")
    break

for X_test, y_test in test_dataloader:
    print(f"Shape of X[N, C, W, H]:{X_test.shape}")
    print(f"Shape of y:{y_test[0]}, {y_test.shape}, {y_test.dtype}")
    break

In [ ]:
device = ("cuda"
          if torch.cuda.is_available()
          else "mps"
          if torch.backends.mps.is_available()
          else "cpu")
print(f"Using {device} device")

class NeunalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.nmodel = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(5*5*64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        # x = nn.Flatten()(x)
        output = self.nmodel(x)
        return output
    
model = NeunalNetwork().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    train_correct, train_loss = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y =X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X.size(0)
        train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    train_loss /= len(dataloader)
    train_correct /= size
    print(f"Train Error: \n Accuracy: {(100*train_correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    return train_loss

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        test_loss /= num_batches
        correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
epochs = 10
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch+1, epochs))
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    test_loss = test(test_dataloader, model, loss_fn)
print('Finished Training')

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')
print(f"Save Pytorch model to model_weights.pth")

In [ ]:
model = NeunalNetwork().to(device)
model.load_state_dict(torch.load("model_weights.pth", weights_only=True))

In [ ]:
model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted , actual = pred[0].argmax(0), y
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x.cpu().numpy().squeeze())
plt.show()